In [90]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid,normalize_0_1
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang, eq_to_seq
import gym, ray
from ray.rllib.agents import ppo
import numpy as np

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
ray.init(ignore_reinit_error=True)


2021-07-08 10:30:25,467	INFO worker.py:745 -- Calling ray.init() again after it has already been called.


In [92]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [105]:
output_lang.n_words

16

In [93]:
# def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:
#     # print("output_sequence ", output_sequence)
#     # print("target_sequence ", output_sequence)
#     magnitude: float = 0.0
#     for x, y in zip(output_sequence, target_sequence):
#         magnitude += abs(x - y)
#     # magnitude /= len(norm_target_seq)
#     return 10 - magnitude


In [94]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:

    if len(output_sequence) != len(target_sequence):
        raise AssertionError("sequence size don't match: " + ','.join(str(e) for e in output_sequence)
 + " | " + ','.join(str(e) for e in target_sequence))

    magnitude: float = 0.0

    combined_seq = np.vstack([output_sequence, target_sequence]) 
    norm_comb_seq = normalize_0_1(combined_seq)

    norm_output_seq = norm_comb_seq[0]
    norm_target_seq = norm_comb_seq[1]

    
    for x, y in zip(norm_target_seq, norm_output_seq):
        magnitude += abs(x - y)#**2

    # magnitude /= len(norm_target_seq)

    return 10 - (magnitude * 100)



In [95]:
compare_sequences([1,2, 3, 100], [1,2,3, 100])

10.0

In [96]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return -MAX_PENALTY_MAGNITUDE

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return -MAX_PENALTY_MAGNITUDE

    return compare_sequences(output_sequence, int_seq)

In [97]:
env = IntegerSequenceEnv({"int_sequence": train[0][0], "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "evaluate": evaluate_candidate_eq})

In [98]:
env.step(5)

(([7, -1, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9, 10]),
 0,
 False,
 {})

In [99]:
env.step(1)

(([7, 3, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9, 10]),
 0,
 False,
 {})

In [100]:
trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
    "env_config": {
        "int_sequence": train[0][0],
        "output_length": 9,
        "input_lang": input_lang,
        "output_lang": output_lang,
        "evaluate": evaluate_candidate_eq
    },
    "num_envs_per_worker": 1,
    # "train_batch_size": 10000,
    "num_workers": 4,
})

2021-07-08 10:30:42,967	INFO trainable.py:101 -- Trainable.setup took 11.866 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-07-08 10:30:42,968	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [101]:

for i in range(10):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(result["episode_reward_mean"])
#    print(result)

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

RayTaskError(ValueError): [36mray::RolloutWorker.par_iter_next()[39m (pid=14697, ip=192.168.2.103)
  File "python/ray/_raylet.pyx", line 501, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/util/iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 333, in gen_rollouts
    yield self.sample()
  File "/usr/local/anaconda3/envs/deeplearning/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 726, in sample
    batches = [self.input_reader.next()]
  File "/usr/local/anaconda3/envs/deeplearning/lib

In [ ]:
result = trainer.train()

In [ ]:
result